# An approximate solver for Burgers' equation

$\newcommand{\wave}{{\mathcal W}}$

As a first example, we return to the inviscid Burgers' equation:

\begin{align} \label{burgers}
q_t + \left(\frac{1}{2}q^2\right)_x & = 0.
\end{align}

Although it is easy to solve the Riemann problem for \eqref{burgers} exactly, it is nevertheless interesting to consider approximate solvers because a numerical scheme does not make use of the full Riemann solution.  Furthermore, Burgers' equation provides a simple setting in which some of the intricacies of more complicated approximate solvers can be introduced.

## How numerical schemes use the Riemann solution
Although we have gone to great pains to determine the full details of each Riemann solution, much of this information is not used in a numerical implementation.  Typically a numerical scheme uses only the value of the flux at $x=0$ to advance the solution.  Equivalently, we can think of the scheme as incorporating only the net impact of all waves traveling to the right (or left) on the adjacent grid points or cells.  It's therefore not necessary to consider the internal structure of a rarefaction wave, but rather only the net change in the solution that the rarefaction wave leads to.  For this reason, we are interested in **approximate solutions that consist entirely of traveling discontinuities**.

## Shock wave solutions

Recall that the exact Riemann solution for \eqref{burgers} consists of a single shock or rarefaction wave.  We have a shock if $q_l>q_r$ and we have a rarefaction if $q_l < q_r$.  In the case of a shock wave, we can simply use the exact solution as our approximation.  We have a single wave of strength $q_r-q_l$ traveling at speed $s=(q_r+q_l)/2$.  In terms of fluxes, the numerical flux is $F=f(q_l)$ if $s>0$ and $F=f(q_r)$ if $s<0$.  In the special case $s=0$ we have a stationary shock, and it must be that $f(q_l)=f(q_r) (=F)$. 

## Rarefaction wave solutions

As discussed above, for numerical purposes it is convenient to approximate a rarefaction wave by a traveling discontinuity.  For Burgers' equation this may seem unnecessary, but for more complicated solvers for systems of equations it will be essential.

We will approximate the effect of the rarefaction wave by a fictitious shock:

$$\wave = q_r-q_l$$

whose speed is given by the Rankine-Hugoniot jump condition:

$$s = \frac{f(q_r)-f(q_l)}{q_r-q_l} = \frac{q_r + q_l}{2}.$$

Recall that this is indeed a weak solution of the Riemann problem.  This fictitious shock is not entropy-satisfying, but as long as it's traveling entirely to the left or entirely to the right, the effect on the numerical solution will be the same as if we used a (entropy-satisfying) rarefaction wave.

The numerical flux is again $F=f(q_l)$ if $s>0$ and $F=f(q_r)$ if $s<0$.

Because we have a scalar equation, both the Roe and HLL approaches will simplify to what we have already described.  But we briefly discuss them here to illustrate the main ideas.

## A Roe solver

Let us consider a linearized solver, in which we replace our nonlinear hyperbolic system with a linearization about some intermediate state $\hat{q}$.  For Burgers' equation, this gives

$$q_t + \hat{q}q_x = 0.$$

This is simply the advection equation with velocity $\hat{q}$.  The solution of the Riemann problem for this equation consists of a wave $\wave = q_r - q_l$ traveling at speed $\hat{q}$.  It remains only to determine the state $\hat{q}$, and thus the wave speed.

The defining feature of a Roe linearization is that it gives the exact solution in case the states $(q_r, q_l)$ lie on a single Hugoniot locus -- i.e., when the solution is a single shock.  We can achieve this by choosing

$$\hat{q} = \frac{q_r + q_l}{2}.$$

This is equivalent to using the approximate solver described already in the sections above.

## An HLL solver

For a scalar equation, it does not make sense fully adopt the HLL approach, which posits a Riemann solution with two waves.  But we can define a solver in the same vein by assuming the solution consists of a single wave, and determining its speed via the requirement of conservation.  In the scalar context, this would more accurately be called a *local Lax-Friedrichs* or *Rusanov* solver.

Since the solution involves only one wave, that wave must carry the entire jump $q_r - q_l$.  In general, a numerical scheme will be conservative if the waves in the Riemann solution satisfy

$$f(q_r) - f(q_l) = \sum_{p=1}^m s^p \wave^p.$$

(*give more explanation of where this comes from*)

For the present simple example, this means

$$f(q_r - f(q_l) = s (q_r - q_l),$$

which is just the Rankine-Hugniot condition and again leads to the speed discussed above.

## Transonic rarefactions

In the approaches above, the solution was approximated by a single wave traveling either to the left or right.  For this scalar problem, this "approximation" is, in fact, an exact weak solution of the Riemann problem.  As discussed already, we do not typically need to worry about the fact that it may be entropy-violating, since the effect on the numerical solution (after averaging) is identical to that of the entropy-satisfying solution.

However, if $q_l < 0 < q_r$, then the true solution is a transonic rarefaction, in which part of the wave travels to the left and part travels to the right.  In this case, the true Riemann solution would lead to changes to both the left and right adjacent cells, whereas an approximate solution with a single wave will only modify one or the other.  This leads to an incorrect numerical solution (on the macroscopic level).  It is therefore necessary to modify the Riemann solver, imposing what is known as an *entropy fix* in the transonic case.

Specifically, we use a solution consisting of two waves, each of which captures the net effect of the corresponding rarefaction wave that appears in the exact solution:

\begin{align}
\wave^1 & = q_m - q_l, & s^1 = \frac{q_l + q_m}{2} \\
\wave^2 & = q_r - q_m, & s^2 = \frac{q_m + q_r}{2}.
\end{align}

Since the middle state of a transonic rarefaction for Burgers' equation is always $q_m = 0$, this simplifies to

\begin{align}
\wave^1 & = - q_l, & s^1 = \frac{q_l}{2} \\
\wave^2 & = q_r, & s^2 = \frac{q_r}{2}.
\end{align}